In [ ]:
# Import necassary libraries
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load datasets
zip_shape_df = gpd.read_file('../data/plz-3stellig.shp', dtype={'plz': str})
df_listings = pd.read_csv('../data/master_listings_20210723.csv')
df_inquiries = pd.read_csv('../data/master_inquiries_20210715.csv')
df_price = pd.read_csv('../data/prices_20210719.csv')

## First map - Germany with properties first-digit postal code area

In [ ]:
zip_shape_df.head()

In [ ]:
zip_shape_df.to_file('../data/zip.geojson', driver='GeoJSON')

In [ ]:
# Create feature.
zip_shape_df = zip_shape_df \
    .assign(first_dig_plz = lambda x: x['plz'].str.slice(start=0, stop=1))

In [ ]:
fig, ax = plt.subplots(figsize=(5, 6.5))

zip_shape_df.plot(
    ax=ax, 
    column='first_dig_plz', 
    categorical=True, 
    legend=True, 
    legend_kwds={'title':'First Digit', 'loc':'lower right', 'bbox_to_anchor':(1.3, 0.0)},
    cmap='tab20',
    alpha=0.9
)

lat, lon = df_listings['lat'], df_listings['lng']

# Scatter the points
plt.scatter(lon, lat, c='black')

right_side = ax.spines["right"]
right_side.set_visible(False)

top_side = ax.spines["top"]
top_side.set_visible(False)

ax.set(
    title='Location Properties in First-Digit-Postal Codes Areas', 
    aspect=1.3,
    facecolor='white'
);

## Preparation for second map

In [ ]:
df_listings.head()

In [ ]:
listings = df_listings[['listing_id', 'zip']]

In [ ]:
inquiry_per_zip = pd.merge(df_inquiries, listings, on='listing_id', how='left')

In [ ]:
inquiry_per_zip = inquiry_per_zip.groupby('zip').agg(['count'])['listing_id'].reset_index()

In [ ]:
inquiry_per_zip['zip'] = inquiry_per_zip.zip.astype(int)

In [ ]:
inquiry_per_zip.rename(columns={'count':'inquiry_count'}, inplace=True)

In [ ]:
df_listings = pd.merge(df_listings, inquiry_per_zip, on='zip', how='left')

## Second map - interactive map with Folium

In [ ]:
# Import necassary libraries
import folium
from folium.plugins import MarkerCluster

In [ ]:
# Create interactive map with folium
map = folium.Map(location=[51.1657, 10.4515])

# add a marker for every record in the filtered data, use a clustered view
marker_cluster = MarkerCluster().add_to(map) # create marker clusters

for i in range(df_listings.shape[0]):
    location = [df_listings['lat'][i],df_listings['lng'][i]]
    tooltip = "Zipcode:{}<br>".format(df_listings["zip"][i])

    folium.Marker(location, 
                  tooltip=tooltip).add_to(marker_cluster)

map    

In [ ]:
# Save map as HTML
map.save('properties_in_germany.html')

## Second interactive map with color